다들 아시는 그 데이터입니다. 네이버 영화리뷰!

In [6]:
%%time
!wget -nc -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

CPU times: user 5.31 ms, sys: 3.66 ms, total: 8.97 ms
Wall time: 113 ms


In [7]:
import codecs
with codecs.open("ratings_train.txt", encoding='utf-8') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]   # 첫번째 col(header) 제외
    
docs = [row[1] for row in data]

이렇게 생겼습니다.

In [8]:
docs

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.',
 '원작의 긴장감을 제대로 살려내지못했다.',
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
 '액션이 없는데도 재미 있는 몇안되는 영화',
 '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?',
 '걍인피니트가짱이다.진짜짱이다♥',
 '볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~',
 '울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해',
 '담백하고 깔끔해서 좋다. 신문기사로만 보다 보면 자꾸 잊어버린다. 그들도 사람이었다는 것을.',
 '취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지',
 'ㄱ냥 매번 긴장되고 재밋음ㅠㅠ',
 '참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다.그냥 까고싶어서 안달난것처럼 보인다',
 '굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐',
 '이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드!!♥',
 '약탈자를 위한 변명, 이라. 저놈들은 착한놈들 절대 아닌걸요.',
 '나름 심오한 뜻도 있는 듯. 그냥 학생이 선생과 놀아나는 영화는 절대 아님',
 '보면서 웃지 않는 건 불가능하다',
 '재미없다 지루하고. 같은 음식 영화인데도 바베트의 만찬하고 넘 차이남....바베트의 만찬은 이야기도 있

일단은 딥러닝이 아니니까 기본적인 전처리가 필요할 거라고 생각해서 간단하게 시도해봅니다.

In [9]:
import re

특별한 내용은 없고 그냥 특수문자만 제외하고 싶었습니다.

In [10]:
onlychar = re.compile(r"[^0-9a-zA-Z가-힣]")

첫번째로는 특수문자들을 없앤 후에, 공백(스페이스)을 기준으로 나눕니다.
그리고 추가적으로 앞뒤로 문장의 시작과 끝을 의미하는 특수문자를 더했습니다.

In [11]:
noSpe=[]
for doc1 in docs:
    oneDoc = onlychar.sub(" ", doc1).split()
    oneDoc.insert(0,"#")
    oneDoc.append("#")
    if len(oneDoc) > 3:
        noSpe.append(oneDoc)

그러면 이런 형태를 구할 수 있습니다.

In [12]:
noSpe

[['#', '아', '더빙', '진짜', '짜증나네요', '목소리', '#'],
 ['#', '흠', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나', '#'],
 ['#', '교도소', '이야기구먼', '솔직히', '재미는', '없다', '평점', '조정', '#'],
 ['#',
  '사이몬페그의',
  '익살스런',
  '연기가',
  '돋보였던',
  '영화',
  '스파이더맨에서',
  '늙어보이기만',
  '했던',
  '커스틴',
  '던스트가',
  '너무나도',
  '이뻐보였다',
  '#'],
 ['#', '막', '걸음마', '뗀', '3세부터', '초등학교', '1학년생인', '8살용영화', '별반개도', '아까움', '#'],
 ['#', '원작의', '긴장감을', '제대로', '살려내지못했다', '#'],
 ['#',
  '별',
  '반개도',
  '아깝다',
  '욕나온다',
  '이응경',
  '길용우',
  '연기생활이몇년인지',
  '정말',
  '발로해도',
  '그것보단',
  '낫겟다',
  '납치',
  '감금만반복반복',
  '이드라마는',
  '가족도없다',
  '연기못하는사람만모엿네',
  '#'],
 ['#', '액션이', '없는데도', '재미', '있는', '몇안되는', '영화', '#'],
 ['#',
  '왜케',
  '평점이',
  '낮은건데',
  '꽤',
  '볼만한데',
  '헐리우드식',
  '화려함에만',
  '너무',
  '길들여져',
  '있나',
  '#'],
 ['#', '걍인피니트가짱이다', '진짜짱이다', '#'],
 ['#', '볼때마다', '눈물나서', '죽겠다90년대의', '향수자극', '허진호는', '감성절제멜로의', '달인이다', '#'],
 ['#', '울면서', '손들고', '횡단보도', '건널때', '뛰쳐나올뻔', '이범수', '연기', '드럽게못해', '#'],
 ['#',
  '담백하고',
  '깔끔해서',
  '좋다',
  '신문기사로만',

In [13]:
from collections import Counter

Word_(n-1), Word_(n) -> 2 Gram
Word_(n-1)을 기준으로 그 다음에 오는 Word_(n)의 종류별로 등장 횟수를 센 후, 이를 이용해서 확률의 형태로 바꿉니다.

In [14]:
def calculate_bigram(sentences):
    bigram = {}
    for s in sentences:
        context = "#"
        for i, w in enumerate(s[1:]):
            if context not in bigram:
                bigram[context] = Counter()
            if bigram[context][w] == 0:
                bigram[context][w] = 1
            else:
                bigram[context][w] += 1
            context = w
    for context in bigram.keys():
        total = sum(bigram[context].values())
        for w in bigram[context]:
            bigram[context][w] /= total
    return bigram

In [15]:
bigram = calculate_bigram(noSpe)

Counter 객체의 most_common 함수를 호출하면, 제한된 숫자의 값들을 정렬된 형태로 확인이 가능합니다.
즉, Word_(n-1)이 앞 글자일 때, Word_(n) 자리에 올 수 있는 것들을 높은 확률 순서대로 출력하는 것입니다.

In [16]:
bigram["#"].most_common(10)

[('정말', 0.01791287339460097),
 ('진짜', 0.014137578921632078),
 ('이', 0.011354647567272152),
 ('아', 0.010189699558470322),
 ('너무', 0.009118235031856294),
 ('그냥', 0.007471487537932721),
 ('이런', 0.006558225827328817),
 ('이거', 0.006191482935668982),
 ('내가', 0.005918223526196948),
 ('이게', 0.0056593461909076525)]

In [17]:
bigram["그냥"].most_common(10)

[('#', 0.011647254575707155),
 ('쓰레기', 0.01136993899057127),
 ('그런', 0.005268996117581808),
 ('좋다', 0.004714364947310039),
 ('다', 0.004714364947310039),
 ('킬링타임용', 0.004437049362174155),
 ('최고', 0.004437049362174155),
 ('시간', 0.004437049362174155),
 ('딱', 0.004159733777038269),
 ('최고다', 0.003882418191902385)]

3 gram용 코드입니다.

In [18]:
def word_ngram(sentence, num_gram):
    ngrams = [tuple(sentence[x:x+num_gram]) for x in range(0, len(sentence)) if len(sentence[x:x+num_gram]) > (num_gram-1)]
    return tuple(ngrams)

In [19]:
fullst=[]
wordgram = 3
for noSpe1 in noSpe:
    ngram3 = word_ngram(noSpe1, wordgram)
    fullst.append(ngram3)

이번에는 일단 3개의 워드로 먼저 나눈 후에 작업을 진행했습니다.

In [20]:
fullst

[(('#', '아', '더빙'),
  ('아', '더빙', '진짜'),
  ('더빙', '진짜', '짜증나네요'),
  ('진짜', '짜증나네요', '목소리'),
  ('짜증나네요', '목소리', '#')),
 (('#', '흠', '포스터보고'),
  ('흠', '포스터보고', '초딩영화줄'),
  ('포스터보고', '초딩영화줄', '오버연기조차'),
  ('초딩영화줄', '오버연기조차', '가볍지'),
  ('오버연기조차', '가볍지', '않구나'),
  ('가볍지', '않구나', '#')),
 (('#', '교도소', '이야기구먼'),
  ('교도소', '이야기구먼', '솔직히'),
  ('이야기구먼', '솔직히', '재미는'),
  ('솔직히', '재미는', '없다'),
  ('재미는', '없다', '평점'),
  ('없다', '평점', '조정'),
  ('평점', '조정', '#')),
 (('#', '사이몬페그의', '익살스런'),
  ('사이몬페그의', '익살스런', '연기가'),
  ('익살스런', '연기가', '돋보였던'),
  ('연기가', '돋보였던', '영화'),
  ('돋보였던', '영화', '스파이더맨에서'),
  ('영화', '스파이더맨에서', '늙어보이기만'),
  ('스파이더맨에서', '늙어보이기만', '했던'),
  ('늙어보이기만', '했던', '커스틴'),
  ('했던', '커스틴', '던스트가'),
  ('커스틴', '던스트가', '너무나도'),
  ('던스트가', '너무나도', '이뻐보였다'),
  ('너무나도', '이뻐보였다', '#')),
 (('#', '막', '걸음마'),
  ('막', '걸음마', '뗀'),
  ('걸음마', '뗀', '3세부터'),
  ('뗀', '3세부터', '초등학교'),
  ('3세부터', '초등학교', '1학년생인'),
  ('초등학교', '1학년생인', '8살용영화'),
  ('1학년생인', '8살용영화', '별반개도'),
  ('8살용영화', '별반개도', '아까움'),
  ('별반

2 gram인 경우와 크기만 다르고 동일한 형태로 구성되어 있습니다.
Word_(n-2), Word_(n-1), Word_(n) -> Word_(n-2), Word_(n-1)들을 기준으로 그 다음에 오는 Word_(n)에 대한 처리를 합니다.

In [21]:
def calculate_ngram(sentences):
    ngram = {}
    for s in sentences:
        for gram1 in s:
            ckey = tuple(gram1[:-1])
            cval = gram1[-1]
            if ckey not in ngram:
                ngram[ckey] = Counter()
            if ngram[ckey][cval] == 0:
                ngram[ckey][cval] = 1
            else:
                ngram[ckey][cval] += 1
    for context in ngram.keys():
        total = sum(ngram[context].values())
        for w in ngram[context]:
            ngram[context][w] /= total
    return ngram

In [22]:
ngram_fullst = calculate_ngram(tuple(fullst))

In [23]:
ngram_fullst["#","흠"].most_common(10)

[('좀', 0.033707865168539325),
 ('뭔가', 0.02247191011235955),
 ('그냥', 0.02247191011235955),
 ('평이', 0.02247191011235955),
 ('걍', 0.02247191011235955),
 ('너무', 0.02247191011235955),
 ('이건', 0.02247191011235955),
 ('포스터보고', 0.011235955056179775),
 ('나름', 0.011235955056179775),
 ('이해하기', 0.011235955056179775)]

In [24]:
ngram_fullst["흠","좀"].most_common(10)

[('그렇던데', 0.3333333333333333),
 ('알잔아요', 0.3333333333333333),
 ('심한데', 0.3333333333333333)]

만든 걸 활용해서 문장 생성을 합니다?

In [25]:
import numpy as np

In [26]:
def gen_sentence(seed=None):
    if seed is not None:
        np.random.seed(seed)
    c = "#"
    sentence = []
    
    while True:
        if c not in bigram:
            break
        words, probs = zip(*[(k, v) for k, v in bigram[c].items()])
        idx = np.argmax(np.random.multinomial(1, probs, (1,)))
        w = words[idx]

        if w == "#":
            break            
        
        sentence.append(" " + w)
        c = w    
    return "".join(sentence).strip()

이런 정도의 문장이 생성됩니다.

In [27]:
gen_sentence(3)

'존 스미스 부부 같이 보면 볼수록 신비한 매력도 없다 스릴러중 이런 영화를 본 영화 스토리 연기 강렬한 폭력미학도 드물거다'

In [34]:
gen_sentence(165)

'한개도 아깝다 이런 다큐멘터리가 세상을 등지고 나이드는모습을 보니 명작이네'